# Decorators

## Material

http://www.artima.com/weblogs/viewpost.jsp?thread=240808

http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/

http://thecodeship.com/patterns/guide-to-python-function-decorators/

## Basic facts

Decorators are basically syntax sugar, which allows encapsulating a function (or an object) withing another function or object which returns it. For example:

In [1]:
def outer(some_func):
    def inner():
        print "before some_func"
        ret = some_func() # 1
        return ret + 1
    return inner

def foo():
    return 1

decorated = outer(foo) # 2
decorated()

before some_func


2

the last 4 lines can be written as:

In [2]:
@outer
def foo2():
    return 1

foo2()


before some_func


2

Definition of a `class` decorator is pretty straightforward: the function is passed to `__init__`, and the arguments to `__call__`:

In [50]:
class entryExit(object):

    def __init__(self, f):
        self.f = f

    def __call__(self, *args):
        print "Entering", self.f.__name__
        self.f(*args)
        print "Exited", self.f.__name__

@entryExit
def func1(x):
    print "inside func1()"
    print x

@entryExit
def func2(x):
    print "inside func2()"
    print x

func1(1)
func2(2)

Entering func1
inside func1()
1
Exited func1
Entering func2
inside func2()
2
Exited func2


NB: you won't be automatically maintain the function's name!

In [51]:
a = func1
print dir(a)
print a.f.__name__


['__call__', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'f']
func1


Using a decorator with arguments on a `class` is less straightforward... in this case, the arguments of the decorator are passed to `__init__`, while the function to `__call__`, that must wrap and return it:

In [52]:
class Wrapper(object):

    def __init__(self, a):
        self.a = a

    def __call__(self, f):
        def wrapped_f(*args):
            self.f = f
            print "Entering", self.f.__name__
            self.f(*args)
            print self.a
            print "Exited", self.f.__name__
        return wrapped_f
    
@Wrapper(1)
def fool(x):
    print "I'm a fool ", x
    
fool("test")

Entering fool
I'm fool  test
1
Exited fool


Entering fool
I'm fool  test
1
Exited fool
